# Configurações inicias 

In [2]:
###############################################################################
###### IMPORTACAO
###############################################################################
from pyspark.sql import SparkSession
import os

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

###############################################################################
###### CONFIGURACOES PARAMETROS PARA INICIAR SESSAO SPARK
###############################################################################
HDP_VERSION = 3  # type: int
SESSION_NAME = "HEDGE"  # type: str
LANGUAGE = "python"

####### Constantes que definem os valores para dimensionamento da sessao spark no yarn


DRIVER_MEMORY = "32g",
DRIVER_CORES = 8,
EXECUTOR_CORES = 16,
EXECUTOR_MEMORY = "32g",
SPARK_CONF = {
"spark.dynamicAllocation.enabled": "true",
"spark.dynamicAllocation.minExecutors" : "16",
"spark.dynamicAllocation.maxExecutors" : "30",
"spark.dynamicAllocation.initialExecutors" : "16",
"spark.dynamicAllocation.schedulerBacklogTimeout" : "2m",
"spark.dynamicAllocation.executorIdleTimeout": "120s",
"spark.shuffle.service.enabled": "true",
"spark.driver.memoryOverhead" : "16g",
"spark.executor.memoryOverhead" : "16g",
"spark.sql.sources.partitionOverwriteMode": "dynamic",
"spark.sql.hive.caseSensitiveInferenceMode": "NEVER_INFER",
"spark.shuffle.service.enabled": "true", 
"spark.shuffle.service.port": "7447", 
"spark.shuffle.service.name": "spark2_shuffle", 
"spark.shuffle.useOldFetchProtocol": "true",
"spark.shuffle.sort.bypassMergeThreshold": "40",
"spark.sql.inMemoryColumnarStorage.compressed": "true",
"spark.sql.inMemoryColumnarStorage.batchSize": "10000"
}


###############################################################################
###### CONSTANTE QUE SERVE PARA DEFINIR PARAMETROS A SEREM UTILIZADOS CONFORME 
###### AMBIENTE (MODELAGEM OU SCORAGE/PRODUCAO) NO QUAL O JOB SERA EXECUTADO 
###############################################################################
AMBIENTE = os.environ["AMBIENTE"] 
if AMBIENTE == "MODELAGEM":
    ENV_VAR = {
        "OWNER_D4F":"",
        "TABELA_ENTRADA":"",
        "AMBIENTE":AMBIENTE,
        "TABLE_DB2_DOMINIO":"",
        "KEYTAB": ""
    }
    !kdestroy
else:
    ENV_VAR = {
        "OWNER_D4F":"",
        "TABELA_ENTRADA":"",
        "TABLE_DB2_DOMINIO":"",
        "KEYTAB": os.environ['KEYTAB']
    }
    
###############################################################################
###### CRIACAO DA SESSAO SPARK 
###############################################################################
spark = Spark(
    session_name= SESSION_NAME, 
    username=  ENV_VAR["KEYTAB"], 
    hdp = HDP_VERSION,
    language = LANGUAGE,
    db2=True,
    spark_conf=SPARK_CONF,
    auth = 'Kerberos',
    env = ENV_VAR
)

NameError: name 'Spark' is not defined

In [90]:
%%spark

import os
import sys
from pyspark.sql import Row
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import col, column, expr, when
from pyspark.sql.types import DecimalType, DateType, DoubleType, IntegerType, LongType, ShortType, StringType, StructField, StructType,TimestampType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### MontaConsultasDB2

In [133]:
%%spark

import pyspark.sql.functions as F

def MontaQuery(key):
    
    cd_mtdl_e = 15
    cd_rgr_e = 225
    gda_nr_crit_rech = 0
    gda_seql_crit_rech = 0
   
    if key == 1:
        
        query_psqC1 = """
        (
        SELECT
        A.IN_RGR_ATV,
        B.CD_CRIT_INRO,
        B.NR_SEQL_CRIT_INRO,
        B.CD_TIP_CTU,
        B.TX_CTU,
        B.IN_CRIT_ATI
        FROM
        "".RGR_INRO_MTDL AS A,
        "".CMPS_RGR_INRO AS B
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_RGR_INRO = {1}
        AND B.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
        AND B.CD_RGR_INRO = A.CD_RGR_INRO
        AND ((B.CD_CRIT_INRO >= {2}
            AND B.NR_SEQL_CRIT_INRO > {3} )
        OR (B.CD_CRIT_INRO > {2} ))
        AND B.NR_SEQL_CMPS = (
        SELECT
        MAX (C.NR_SEQL_CMPS)
        FROM
        "".CMPS_RGR AS C
        WHERE
        C.CD_MTDL_ANL_CRD = {0}
        AND C.CD_RGR_INRO = {1}
        AND C.TS_INC_VGC <> '0001-01-01-00.00.00.000000')
        ORDER BY
        B.IN_CRIT_ATI DESC,
        B.CD_CRIT_INRO ,
        B.NR_SEQL_CRIT_INRO
        --FETCH FIRST 72 ROWS ONLY
         ) as query_psqC1
         """.format(cd_mtdl_e,cd_rgr_e,gda_nr_crit_rech,gda_seql_crit_rech)


        query_psqC1 = spark.read.format("jdbc") \
                                .option("URL",URL) \
                                .option("driver",JCN) \
                                .option("dbtable",query_psqC1) \
                                .option("user",DB2_USER) \
                                .option("password",DB2_PASSWORD) \
                                .option('KeepAliveTimeout','10') \
                                .load()
           
        return query_psqC1
    
    elif key == 2:
        query_psqC2 = """
        (
        SELECT
        A.IN_RGR_ATV,
        B.NR_SEQL_CMPS,
        B.NR_SEQL_CRIT_INRO,
        B.CD_TIP_CTU,
        B.TX_CTU,
        B.IN_CRIT_ATI
        FROM
        "".RGR_INRO_MTDL AS A,
        "".CMPS_RGR_INRO AS B
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_RGR_INRO = {1}
        AND B.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
        AND B.CD_RGR_INRO = A.CD_RGR_INRO
        AND B.CD_CRIT_INRO = {2}
        AND B.NR_SEQL_CMPS = (
        SELECT
            MAX (C.NR_SEQL_CMPS)
        FROM
            "".CMPS_RGR AS C
        WHERE
            C.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
            AND C.CD_RGR_INRO = A.CD_RGR_INRO
            AND C.TS_INC_VGC <> '0001-01-01-00.00.00.000000')
        ORDER BY
        B.NR_SEQL_CRIT_INRO
        --FETCH FIRST 72 ROWS ONLY
        ) as query_psqC2
        """.format(aux_cd_mtdl,aux_cd_rgr,cd_crit_e)
        
        query_psqC2 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC2) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC2
    
    elif key == 3:
        
        query_psqC3 = """
        (
        SELECT
        A.IN_RGR_ATV,
        B.CD_CRIT_INRO,
        B.NR_SEQL_CRIT_INRO,
        B.CD_TIP_CTU,
        B.TX_CTU,
        B.IN_CRIT_ATI
        FROM
        "".RGR_INRO_MTDL AS A,
        "".CMPS_RGR_INRO AS B,
        "".CMPS_RGR AS C
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_RGR_INRO = {1}
        AND B.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
        AND B.CD_RGR_INRO = A.CD_RGR_INRO
        AND C.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
        AND C.CD_RGR_INRO = A.CD_RGR_INRO
        AND C.NR_SEQL_CMPS = B.NR_SEQL_CMPS
        AND C.TS_INC_VGC = '0001-01-01-00.00.00.000000'
        AND ((B.CD_CRIT_INRO >= {2}
            AND B.NR_SEQL_CRIT_INRO > {2} )
        OR (B.CD_CRIT_INRO > {2} ))
        GROUP BY
        A.IN_RGR_ATV,
        B.CD_CRIT_INRO,
        B.NR_SEQL_CRIT_INRO,
        B.CD_TIP_CTU,
        B.TX_CTU,
        B.IN_CRIT_ATI
        ORDER BY
        B.CD_CRIT_INRO ,
        B.NR_SEQL_CRIT_INRO
        --FOR FETCH ONLY
        ) as query_psqC3
        """.format(aux_cd_mtdl,aux_cd_rgr,cd_crit_e)

        query_psqC3 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC3) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC3
    
    
    elif key == 4:
        
        query_psqC4 = """
        (
        SELECT
        COUNT(*) AS CONTAGEM_GERAL
        FROM
        "".CMPS_RGR A
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_RGR_INRO = {1}
        AND A.TS_INC_VGC = '0001-01-01-00.00.00.000000'
        ) as query_psqC4
        """.format(aux_cd_mtdl,aux_cd_rgr)

        query_psqC4 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC4) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC4
    
    
    
    
    elif key == 5:
        
        query_psqC5 = """
        (
        SELECT
        DISTINCT CD_MTDL_ANL_CRD
        FROM
        "".CMPS_RGR_INRO
        WHERE
        CD_MTDL_ANL_CRD > {0}
        ORDER BY
        CD_MTDL_ANL_CRD
        ) as query_psqC5
        """.format(aux_cd_mtdl)

        query_psqC5 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC5) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC5
    
    
    elif key == 6:
        
        query_psqC6 = """
        (
        SELECT
        DISTINCT A.CD_MTDL_ANL_CRD
        FROM
        "".CMPS_RGR_INRO A,
        "".CMPS_RGR B
        WHERE
        A.CD_MTDL_ANL_CRD > {0}
        AND A.CD_MTDL_ANL_CRD = B.CD_MTDL_ANL_CRD
        AND A.CD_RGR_INRO = B.CD_RGR_INRO
        AND A.NR_SEQL_CMPS = B.NR_SEQL_CMPS
        AND A.NR_SEQL_CMPS = (
        SELECT
            MAX (C.NR_SEQL_CMPS)
        FROM
            "".CMPS_RGR C
        WHERE
            C.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
            AND C.CD_RGR_INRO = A.CD_RGR_INRO )
        AND B.TS_INC_VGC <> '0001-01-01-00.00.00.000000'
        ORDER BY
        A.CD_MTDL_ANL_CRD
        ) as query_psqC6
        """.format(aux_cd_mtdl)

        query_psqC6 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC6) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC6
    
    
    
    elif key == 7:
        
        query_psqC7 = """
        (
        SELECT
        DISTINCT CD_MTDL_ANL_CRD
        FROM
        "".CMPS_RGR
        WHERE
        CD_MTDL_ANL_CRD > {0}
        AND TS_INC_VGC = '0001-01-01-00.00.00.000000'
        ORDER BY
        CD_MTDL_ANL_CRD
        ) as query_psqC7
        """.format(aux_cd_mtdl)

        query_psqC7 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC7) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC7
    
    
    elif key == 8:
        
        query_psqC8 = """
        (
        SELECT
        DISTINCT A.CD_MTDL_ANL_CRD
        FROM
        "".CMPS_RGR_INRO A,
        "".CMPS_RGR B
        WHERE
        A.CD_MTDL_ANL_CRD > {0}
        AND A.CD_MTDL_ANL_CRD = B.CD_MTDL_ANL_CRD
        AND A.CD_RGR_INRO = B.CD_RGR_INRO
        AND A.NR_SEQL_CMPS = B.NR_SEQL_CMPS
        AND A.NR_SEQL_CMPS <> (
        SELECT
            MAX (C.NR_SEQL_CMPS)
        FROM
            "".CMPS_RGR C
        WHERE
            C.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
            AND C.CD_RGR_INRO = A.CD_RGR_INRO)
        AND B.TS_INC_VGC <> '0001-01-01-00.00.00.000000'
        ORDER BY
        A.CD_MTDL_ANL_CRD
        ) as query_psqC8
        """.format(aux_cd_mtdl)

        query_psqC8 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC8) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC8
    
    
    elif key == 9:
        
        query_psqC9 = """
        (
        SELECT
        A.IN_RGR_ATV,
        B.CD_CRIT_INRO,
        B.NR_SEQL_CRIT_INRO,
        B.CD_TIP_CTU,
        B.TX_CTU,
        B.IN_CRIT_ATI
        FROM
        "".RGR_INRO_MTDL AS A,
        "".CMPS_RGR_INRO AS B
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_RGR_INRO = {1}
        AND B.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
        AND B.CD_RGR_INRO = A.CD_RGR_INRO
        AND ((B.CD_CRIT_INRO >= {2}
            AND B.NR_SEQL_CRIT_INRO > {3})
        OR (B.CD_CRIT_INRO > {2}))
        AND B.NR_SEQL_CMPS = (
        SELECT
            MAX (C.NR_SEQL_CMPS)
        FROM
            "".CMPS_RGR AS C
        WHERE
            C.CD_MTDL_ANL_CRD = {0}
            AND C.CD_RGR_INRO = {1}
            AND C.TS_INC_VGC <> '0001-01-01-00.00.00.000000')
        AND A.IN_RGR_ATV = 'S'
        AND B.IN_CRIT_ATI = 'S'
        ORDER BY
        B.CD_CRIT_INRO,
        B.NR_SEQL_CRIT_INRO
        --FETCH FIRST 72 ROWS ONLY
        ) as query_psqC9
        """.format(cd_mtdl_e,cd_rgr_e,gda_nr_crit_rech,gda_seql_crit_rech)

        query_psqC9 =    spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC9) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
    
        return query_psqC9
    
    
    
    elif key == 10:
        
        query_psqC10 = """
        (
        SELECT
        MAX (A.NR_SEQL_CRIT_INRO) AS NR_SEQL_CRIT_INRO
        FROM
        "".CMPS_RGR_INRO A,
        "".CMPS_RGR B
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_MTDL_ANL_CRD = B.CD_MTDL_ANL_CRD
        AND A.CD_RGR_INRO = {1}
        AND A.CD_RGR_INRO = B.CD_RGR_INRO
        AND A.CD_CRIT_INRO = {2}
        AND A.NR_SEQL_CMPS = B.NR_SEQL_CMPS
        AND B.TS_INC_VGC = '0001-01-01-00.00.00.000000'
        ) as query_psqC10
        """.format(aux_cd_mtdl,aux_cd_rgr,cd_crit_e)
    
        query_psqC10 = spark.read.format("jdbc") \
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC10) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()  
        return query_psqC10
    
    
    elif key == 11:
            
        query_psqC11 = """
        (
        SELECT
        B.CD_TIP_CTU,
        B.TX_CTU,
        B.IN_CRIT_ATI
        FROM
        "".RGR_INRO_MTDL AS A,
        "".CMPS_RGR_INRO AS B
        WHERE
        A.CD_MTDL_ANL_CRD = {0}
        AND A.CD_RGR_INRO = {1}
        AND B.CD_MTDL_ANL_CRD = A.CD_MTDL_ANL_CRD
        AND B.CD_RGR_INRO = A.CD_RGR_INRO
        AND B.CD_CRIT_INRO = {2}
        AND B.NR_SEQL_CRIT_INRO = {3}
        AND B.NR_SEQL_CMPS = {4}
        ) as query_psqC11
        """.format(aux_cd_mtdl,aux_cd_rgr,cd_crit_e,gda_nr_seql_crit_loc,gda_nr_seql_cmps)
    
        query_psqC11 = spark.read.format("jdbc")\
                                 .option("URL",URL) \
                                 .option("driver",JCN) \
                                 .option("dbtable",query_psqC11) \
                                 .option("user",DB2_USER) \
                                 .option("password",DB2_PASSWORD) \
                                 .option('KeepAliveTimeout','10') \
                                 .load()
        
        return query_psqC11

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### PesquisaMetodologia

In [134]:
%%spark

from pyspark.sql import *
from pyspark.sql.functions import *

def pesquisaMetodologia(df,psq):
        '''
        Funcao responsavel por chamar as metodologias, vigente, pendente, histórico e regra pendente
        
        Arguments:
            df {DataFrame} -- dataframe criado a partir da carga do arquivo contendo os registros dos clientes
        Keyword Arguments:
            None
        Returns:
            {DataFrame}
        See:
            funcao acionada: 
                None
            chamada pela funcao:
                def RotinaPrincipal
        '''
        
        w=Window.orderBy(lit(1))
        
        if psq == "mtld":
            
            query_psq = MontaQuery(5)
            query_psq = query_psq.withColumn("rn",row_number().over(w)-1)            
            
            pesquisaMetodologiaMtldDF = df.withColumn("rn",row_number().over(w)-1)
            
            pesquisaMetodologiaMtldDF = pesquisaMetodologiaMtldDF.join(query_psq,["rn"],"left")\
                                                                 .withColumn("cd_mtdl_anl_aux",col("CD_MTDL_ANL_CRD"))\
                                                                 .drop("CD_MTDL_ANL_CRD")\
                                                                 .withColumn("qt_reg",col("rn"))\
                                                                 .drop(col("rn"))
            
            return pesquisaMetodologiaMtldDF 
        
        if psq == "vgt":
            
            query_psq = MontaQuery(6)
            query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
            
            pesquisaMetodologiaVgtDF = df.withColumn("rn",row_number().over(w)-1)
            pesquisaMetodologiaVgtDF = pesquisaMetodologiaVgtDF.join(query_psq,["rn"],"left")\
                                                               .withColumn("cd_mtdl_anl_aux",col("CD_MTDL_ANL_CRD"))\
                                                               .drop("CD_MTDL_ANL_CRD")\
                                                               .withColumn("qt_reg",col("rn"))\
                                                               .drop(col("rn"))
            
            return pesquisaMetodologiaVgtDF
        
        if psq == "pnd":
            
            query_psq = MontaQuery(7)
            query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
            
            pesquisaMetodologiaPndDF = df.withColumn("rn",row_number().over(w)-1)
            pesquisaMetodologiaPndDF = pesquisaMetodologiaPndDF.join(query_psq,["rn"],"left")\
                                                               .withColumn("cd_mtdl_anl_aux",col("CD_MTDL_ANL_CRD"))\
                                                               .drop("CD_MTDL_ANL_CRD")\
                                                               .withColumn("qt_reg",col("rn"))\
                                                               .drop(col("rn"))
                
            return pesquisaMetodologiaPndDF
        
        if psq == "hst":
            
            query_psq = MontaQuery(8)
            query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
            
            pesquisaMetodologiaHstDF = df.withColumn("rn",row_number().over(w)-1)
            pesquisaMetodologiaHstDF = pesquisaMetodologiaHstDF.join(query_psq,["rn"],"left")\
                                                               .withColumn("cd_mtdl_anl_aux",col("CD_MTDL_ANL_CRD"))\
                                                               .drop("CD_MTDL_ANL_CRD")\
                                                               .withColumn("qt_reg",col("rn"))\
                                                               .drop(col("rn"))           
            
            return pesquisaMetodologiaHstDF
        
        if psq == "rgr":
            
            query_psq = MontaQuery(4)
            query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
            
            pesquisaMetodologiaRgrDF = df.withColumn("rn",row_number().over(w)-1)
            pesquisaMetodologiaRgrDF = pesquisaMetodologiaRgrDF.join(query_psq,["rn"],"left").drop("rn")\
                                                               .withColumn("qt_reg",F.when(col("CONTAGEM_GERAL").isnull(),lit(0))\
                                                               .otherwise(col("CONTAGEM_GERAL"))\
                                                                          )\
                                                               .drop(col("CONTAGEM_GERAL"))
            
            return pesquisaMetodologiaRgrDF

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### PesquisaCriterio2

In [135]:
%%spark

from pyspark.sql import *
from pyspark.sql.functions import *

def pesquisaCriterio2(df):
        '''
        Metodo traduzido da subrotina
        
        Arguments:
            df {DataFrame} -- dataframe criado a partir da carga do arquivo contendo os registros dos clientes
        Keyword Arguments:
            None
        Returns:
            {DataFrame}
        See:
            funcao acionada: 
                None
            chamada pela funcao:
                def RotinaPrincipal
        '''
        
        w=Window.orderBy(lit(1))
        
        query_psq  = MontaQuery(11)
        query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
        
        pesquisaCriterio2DF = df.withColumn("rn",row_number().over(w)-1)\
                                .withColumn("qt_reg",col("qt_reg") + 1 )\
                                .withColumn("cd_crit",col("aux_cd_crit"))\
                                .withColumn("nr_seql_crit",col("gda_nr_seql_crit_loc"))
        
        pesquisaCriterio2DF = pesquisaCriterio2DF.join(query_psq, ["rn"], "left")\
                                                 .drop("rn")\
                                                 .withColumn("cd_tip_ctu", F.col("CD_TIP_CTU"))\
                                                 .withColumn("tx_ctu",     F.col("TX_CTU"))\
                                                 .withColumn("vl_flo_crit",F.col("TX_CTU"))\
                                                 .withColumn("in_crit_atv",F.col("IN_CRIT_ATI"))\
                                                 .drop("IN_CRIT_ATI")
        

        
        return pesquisaCriterio2DF

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### PesquisaCriterioPendente

In [136]:
%%spark


from pyspark.sql import *
from pyspark.sql.functions import *

def pesquisarCriterioPend(df):
        '''
        Metodo traduzido da rotina
        
        Arguments:
            df {DataFrame} -- dataframe criado a partir da carga do arquivo contendo os registros dos clientes
        Keyword Arguments:
            None
        Returns:
            {DataFrame}
        See:
            funcao acionada: 
                None
            chamada pela funcao:
                def RotinaPrincipal
        '''
        
        # constante 
        
        w=Window.orderBy(lit(1))
        
        # expressao logica
        
        EXPR_IN_CARREGA_TAB = expr("in_ret_bxd = 'S' or gda_in_rgr_atv = 'S' and gda_crit_atv = 'S' ")
        
        # Inicialização 
        
        query_psq  = MontaQuery(3)
        query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
        
        pesquisarCriterioPendDF = df.withColumn("rn",row_number().over(w)-1)
        
        
        pesquisarCriterioPendDF =  pesquisarCriterioPendDF.join(query_psq, ["rn"], "left")\
                                                          .drop("rn")\
                                                          .withColumn("gda_in_rgr_atv",  F.col("IN_RGR_ATV"))\
                                                          .drop("IN_RGR_ATV")\
                                                          .withColumn("gda_cd_crit",     F.col("CD_CRIT_INRO"))\
                                                          .drop("CD_CRIT_INRO")\
                                                          .withColumn("gda_nr_seql_crit",F.col("NR_SEQL_CRIT_INRO"))\
                                                          .drop("NR_SEQL_CRIT_INRO")\
                                                          .withColumn("gda_cd_tip_ctu",  F.col("CD_TIP_CTU"))\
                                                          .drop("CD_TIP_CTU")\
                                                          .withColumn("gda_tx_ctu",      F.col("TX_CTU"))\
                                                          .drop("TX_CTU")\
                                                          .withColumn("gda_in_crit_atv", F.col("IN_CRIT_ATI"))\
                                                          .drop("IN_CRIT_ATI")\
                                                          .withColumn("in_carregaTab",   F.when(EXPR_IN_CARREGA_TAB,lit(True))\
                                                                                          .otherwise(lit(False))\
                                                                     )
                                     
            
        
        return pesquisarCriterioPendDF

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### PesquisaSelecaoCriterio

In [137]:
%%spark


from pyspark.sql import *
from pyspark.sql.functions import *

def pesquisarSeqlCrit(df):
        '''
        Metodo traduzido da rotina
        
        Arguments:
            df {DataFrame} -- dataframe criado a partir da carga do arquivo contendo os registros dos clientes
        Keyword Arguments:
            None
        Returns:
            {DataFrame}
        See:
            funcao acionada: 
                None
            chamada pela funcao:
                def RotinaPrincipal
        '''
        
        # constante 
        
        w=Window.orderBy(lit(1))
        
        # Inicialização 
        
        query_psq  = MontaQuery(10)
        query_psq = query_psq.withColumn("rn",row_number().over(w)-1)
        
        pesquisarSeqlCritDF = df.withColumn("rn",row_number().over(w)-1)
        
        
        pesquisarSeqlCritDF =  pesquisarSeqlCritDF.join(query_psq, ["rn"], "left")\
                                                  .withColumn("qtd_reg",lit(1))\
                                                  .withColumn("nr_seql_crit",F.when(col("NR_SEQL_CRIT_INRO").isnull(),lit(0))\
                                                                              .otherwise(col("NR_SEQL_CRIT_INRO"))\
                                                              )\
                                                  .drop("NR_SEQL_CRIT_INRO")
        
                                                              
        return pesquisarSeqlCritDF
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### RotinaPrincipal

In [138]:
%%spark

from pyspark.sql import *
from pyspark.sql.functions import *

def rotina_Principal(df):
        '''
        Acionada tem o objetivo chamar as rotinas que pesquisam os critérios e as metodologias.
        
        Arguments:
            df {DataFrame} -- dataframe criado a partir da carga do arquivo contendo os registros dos clientes
               
        Keyword Arguments:
            None
        Returns:
            {DataFrame}
        See:
            funcao acionada: 
                def MontaQuery
                def pesquisarCriterioPend
                def pesquisaMetodologia
            chamada pela funcao:
                def FuncPrincipal
        '''

        #1 Verifica as condições para criação da cod das query
        #2 Realiza a montagem das query e a execução no banco de dados retornando um dataframe spark atraves da função MONTAQUERY 
        #3 IN_MONTAQUERY: Construção dos parametros para as funções de construções e execução das query no banco de dados já formatadas conforme função
        #4 MONTAQUERY: Refere-se a função MONTAQUERY onde a mesma formata determinadas query e executa as mesmas no banco de dados o resultado é transformado em dataframe spark 
        #5 EXPR_IN_PESQUISA_CRITERIO_1_9: Refere-se ao indicador para execução da query com chave 1 mas executando a query de chave 9 
        
        
        # constantes
        
        w=Window.orderBy(lit(1))

        
        # expressoes logicas
        
        EXPR_IN_RET_BXD = expr("in_ret_bxd = 'S'")
        EXPR_COD_FUNC_1 = expr("cd_fuc = 1")
        EXPR_IN_PESQUISA_CRITERIO_1 = EXPR_COD_FUNC_1 & EXPR_IN_RET_BXD
        EXPR_IN_PESQUISA_CRITERIO_1_9 = EXPR_COD_FUNC_1 & ~EXPR_IN_RET_BXD
        EXPR_IN_MONTAQUERY_2 = expr("cd_fuc = 2")
        EXPR_IN_MONTAQUERY_3 = expr("cd_fuc = 3")
        EXPR_IN_MONTAQUERY_4 = expr("cd_fuc = 4")
        EXPR_IN_MONTAQUERY_5 = expr("cd_fuc = 5")
        EXPR_IN_MONTAQUERY_6 = expr("cd_fuc = 6")
        EXPR_IN_MONTAQUERY_7 = expr("cd_fuc = 7")
        EXPR_IN_MONTAQUERY_8 = expr("cd_fuc = 8")
        EXPR_IN_MONTAQUERY_9 = expr("cd_fuc = 9")
        EXPR_IN_MONTAQUERY_10 = expr("cd_fuc = 10")


        # inicialização 
        
        rotina_PrincipalDF = df.withColumn("Cod_MontaQuery", F.when(EXPR_IN_MONTAQUERY_2,lit(2))\
                                                              .when(EXPR_IN_MONTAQUERY_3,lit(3))\
                                                              .when(EXPR_IN_MONTAQUERY_4,lit(4))\
                                                              .when(EXPR_IN_MONTAQUERY_5,lit(5))\
                                                              .when(EXPR_IN_MONTAQUERY_6,lit(6))\
                                                              .when(EXPR_IN_MONTAQUERY_7,lit(7))\
                                                              .when(EXPR_IN_MONTAQUERY_8,lit(8))\
                                                              .when(EXPR_IN_MONTAQUERY_9,lit(9))\
                                                              .when(EXPR_IN_MONTAQUERY_10,lit(10))\
                                                              .otherwise(lit(0))\
                                          )\
                               .withColumn("Cod_pesquisa_criterio",F.when(EXPR_IN_PESQUISA_CRITERIO_1_9,lit(9))\
                                                                    .when(EXPR_IN_PESQUISA_CRITERIO_1,lit(1))\
                                                                    .otherwise(lit(0))\
                                          )\
                               .withColumn("rn",row_number().over(w)-1)
          
        
        cod_pesquisa_criterio = rotina_PrincipalDF.limit(1).select("Cod_pesquisa_criterio").collect()[0]["Cod_pesquisa_criterio"]
        cod_query = rotina_PrincipalDF.limit(1).select("Cod_MontaQuery").collect()[0]["Cod_MontaQuery"]
        
        
        if cod_pesquisa_criterio == 1 or cod_pesquisa_criterio == 9:
            
            QueryMontadaDF = MontaQuery(cod_pesquisa_criterio)
            QueryMontadaDF = QueryMontadaDF.withColumn("rn",row_number().over(w)-1)
            
            rotina_PrincipalDF = rotina_PrincipalDF.join(QueryMontadaDF,["rn"],"left")\
                                                   .drop(col("rn"))

        
        
        
        elif cod_query == 2 or cod_query == 4:
            
            
            QueryMontadaDF = MontaQuery(cod_query)
            QueryMontadaDF = QueryMontadaDF.withColumn("rn",row_number().over(w)-1)
            
            rotina_PrincipalDF = rotina_PrincipalDF.join(QueryMontadaDF,["rn"],"left")\
                                                   .drop(col("rn"))
            
        
        elif cod_query == 3:
            
            rotina_PrincipalDF = pesquisarCriterioPend(rotina_PrincipalDF)                                  
                                           
                                           
        elif cod_query == 5:
            
            rotina_PrincipalDF = pesquisaMetodologia(rotina_PrincipalDF,"mtld")
            
        
        elif cod_query == 6:
            
            rotina_PrincipalDF = pesquisaMetodologia(rotina_PrincipalDF,"vgt")
            
        elif cod_query == 7:
            
            rotina_PrincipalDF = pesquisaMetodologia(rotina_PrincipalDF,"pnd")
            
        elif cod_query == 8:
            
            rotina_PrincipalDF = pesquisaMetodologia(rotina_PrincipalDF,"hst")
            
        elif cod_query == 9:
            
            rotina_PrincipalDF = pesquisarSeqlCrit(rotina_PrincipalDF)
        
        elif cod_query == 10:
            
            rotina_PrincipalDF = pesquisaMetodologia(rotina_PrincipalDF,"rgr")    
            

        
        return rotina_PrincipalDF

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [191]:
# ENCERRANDO A SESSÃO 

%spark cleanup